![logo](rigetti_logo_rgb_teal_notebook.png)

# Improving the performance of Max-Cut QAOA with Quil-T

This notebook shows how to run the **Quantum Approximate Optimization Algorithm** (QAOA) on Aspen-8, using features of **Quil-T** to disable global fencing on CZ gates and, in doing so, improve the fidelity of the algorithm. This extends the ``MaxCutQAOA.ipynb`` notebook provided to our Quantum Cloud Services (QCS) users.

A fence is a barrier in time used to sequence operations at the pulse control level. To minimize crosstalk on 2-qubit (2Q) gates, global fencing is normally enabled, meaning that each 2Q pulse sequence is applied with no other operations occurring on the QPU. While this maximizes the fidelity of any single 2Q gate, it also limits the total 2Q gate depth that can be achieved within the coherence time of the system. For shallow circuits less than 10x 2Q gates, this is certainly the right choice. However, for deep circuits greater than about 20x 2Q gates, it can be that disabling fencing provides a net improvement in fidelity for specific applications. We will demonstrate this effect in the context of a Max-Cut problem run using QAOA.

In [ ]:
import itertools
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from tqdm import tqdm

from typing import Any, Dict, List, Optional, Set, Tuple

from pyquil import get_qc, Program
from pyquil.api import QuantumComputer, QPUCompiler, WavefunctionSimulator
from pyquil.gates import H, MEASURE, RESET
from pyquil.paulis import exponential_map, PauliSum, sX, sZ
from pyquil.quilbase import DefCalibration, Fence, FenceAll

## Consolidated routines from the basic Max-Cut QAOA demonstration

Here we consolidate the routines for generating Max-Cut QAOA problems and programs, running and plotting the algorithm result. For a detailed explanation of these routines, refer to the ``MaxCutQAOA.ipynb`` notebook.

In [ ]:
def generate_ising_graph(edges: List[Tuple[int, int]], seed: int) -> nx.Graph:
    np.random.seed(seed)
    graph = nx.from_edgelist(edges)
    weights:  np.ndarray = np.random.uniform(low=-1.0, high=+1.0, size=graph.number_of_edges())
    nx.set_edge_attributes(graph, {e: {'w': w} for e, w in zip(graph.edges, weights)})
    return graph

def bitstring_cut_weight(b: List[List[int]], graph: nx.Graph) -> dict:
    cost = 0
    inverse_map = {qubit: idx for idx, qubit in enumerate(list(graph.nodes))}
    for q0, q1 in graph.edges():
        cost += graph.get_edge_data(q0, q1)['w'] * (-1) ** int(b[inverse_map[q0]] != b[inverse_map[q1]])
    return cost

def maxcut_qaoa_program(graph: nx.Graph) -> Program:
    cost_ham = PauliSum([sZ(i) * sZ(j) * graph.get_edge_data(i, j)['w'] for i, j in graph.edges])
    driver_ham = PauliSum([sX(i) for i in graph.nodes])

    p = Program(RESET())
    beta = p.declare('beta', 'REAL')
    gamma = p.declare('gamma', 'REAL')
    ro = p.declare('ro', 'BIT', len(graph.nodes))

    p.inst(H(qubit) for qubit in list(graph.nodes))
    p.inst(exponential_map(term)(gamma) for term in cost_ham)
    p.inst(exponential_map(term)(beta) for term in driver_ham)

    p.inst(MEASURE(qubit, ro[idx]) for idx, qubit in enumerate(list(graph.nodes)))

    return p

def plot_landscape(landscape: np.ndarray):
    width = landscape.shape[0]
    max_x, max_y = (np.argmax(landscape) % width, np.argmax(landscape) // width)
    plt.imshow(landscape, extent=[0, np.pi, np.pi, 0])
    plt.plot((max_x + 0.5) * np.pi / width, (max_y + 0.5) * np.pi / width, 'ro')
    plt.colorbar()
    plt.xlabel('beta (radians)')
    plt.ylabel('gamma (radians)')
    plt.title('Max-Cut QAOA Landscape')
    plt.show()

## Patching the ``pyquil`` instruction set architecture to only use CZ

To disable global fencing on CZ gates, we must first take steps to ensure that the QAOA ansatz circuit is compiled to only use the CZ gate. This routine patches the ``pyquil`` instruction set architecture used by the compiler to only support CZ in the set of 2Q instructions. This approach is equally valid to force compilation to CPHASE or XY gates; an exercise that we leave to the reader.

In [ ]:
def patch_pyquil_isa_to_cz(qc: QuantumComputer):
    pyquil_device_isa_dict = qc.device.get_isa().to_dict()

    def filter_operations_dict_cz(operations_dict: Dict[str, Any]) -> Dict[str, Any]:
        return {
            "gates": [
                gate_dict
                for gate_dict in operations_dict["gates"]
                if gate_dict["operator"] == "CZ"
            ]
        }

    pyquil_device_isa_dict_1q = pyquil_device_isa_dict["1Q"]

    pyquil_device_isa_dict_2q = {}
    for edge_id_str, operations_dict in pyquil_device_isa_dict["2Q"].items():
        filtered_operations_dict = filter_operations_dict_cz(operations_dict)
        if len(filtered_operations_dict["gates"]) != 0:
            pyquil_device_isa_dict_2q[edge_id_str] = filtered_operations_dict

    pyquil_device_isa_dict = {"1Q": pyquil_device_isa_dict_1q, "2Q": pyquil_device_isa_dict_2q}
    
    qc.compiler.target_device.isa = pyquil_device_isa_dict

## Generate new calibrations that disable global fencing using Quil-T

Quil-T extends Quil programs with the concept of *calibrations*. A calibration defines the pulse-level control sequence for a specific native instruction at a specific site. For example, the Quil operation "CZ 31 32" will have a calibration for that gate (CZ) at that site (the edge 31-32), written in Quil-T as "DEFCAL CZ 31 32".

To disable global fencing on CZ gates, we must generate new calibrations using Quil-T that modify the pulse-level control sequence to replace global fencing directives ("FENCE_ALL") with fencing local to just the 2 qubits being operated upon ("FENCE \<q0\> \<q1\>"). We achieve this by first retreiving the current calibrations as updated at the last retune, calling `qc.compiler.get_calibration_program()`, and modify them in the stated way.

In [ ]:
def disable_global_fencing_on_cz(qc: QuantumComputer) -> Program:
    quilt_calibration_program = qc.compiler.get_calibration_program()

    quilt_calibrations_nofence = []
    for calibration in quilt_calibration_program.calibrations:
        if isinstance(calibration, DefCalibration):
            if calibration.name == "CZ":
                updated_instrs = []
                for instr in calibration.instrs:
                    if isinstance(instr, FenceAll):  # replace FenceAll
                        updated_instrs.append(Fence(calibration.qubits))
                    else:
                        updated_instrs.append(instr)
                quilt_calibrations_nofence.append(
                    DefCalibration(calibration.name, calibration.parameters, calibration.qubits, updated_instrs)
                )

    return Program(quilt_calibrations_nofence)

## Run the landscape, with or without disabling global fencing using Quil-T

We provide the complete sequence for obtaining a ``pyquil`` quantum computer, modifying the instruction set architecture to only use CZ gates, compiling the QAOA ansatz circuit to native gates, and optionally add the updated Quil-T calibrations needed to disable global fencing on CZ. By setting the ``disable_global_fencing`` flag, we can see the comparative effect on noise and QAOA performance.

**Note:** To provide a reasonable demonstration, we isolate all operational edges within the right-hand two octogons of the Aspen-8 chip.

In [ ]:
def run_maxcut_qaoa_landscape(qc_name: str, disable_global_fencing: bool = False, width: int = 20, 
                              shots: int = 1000, seed: int = 0) -> Tuple[np.ndarray, Program, Program]:
    
    qc = get_qc(qc_name)
    patch_pyquil_isa_to_cz(qc)
    qc_device_isa_restricted_edges = qc.compiler.target_device.isa["2Q"]

    edges = []
    for edge in qc.device.get_isa().edges:
        if not edge.dead:
            q0, q1 = edge.targets
            if q0 >= 30 and q1 >= 30:
                if f"{q0}-{q1}" in qc_device_isa_restricted_edges:
                    edges.append((q0, q1))
    
    graph = generate_ising_graph(edges, seed)
    
    program = maxcut_qaoa_program(graph)
    program.wrap_in_numshots_loop(shots)
    
    native_program = qc.compiler.quil_to_native_quil(program)

    if disable_global_fencing:
        native_program += disable_global_fencing_on_cz(qc)

    executable = qc.compiler.native_quil_to_executable(native_program)

    costs = []
    angle_range = np.linspace(0, np.pi, width)
    landscape = list(itertools.product(angle_range, angle_range))
    for beta, gamma in tqdm(landscape):
        memory_map = {'beta': [beta], 'gamma': [gamma]}
        bitstrings = qc.run(executable, memory_map=memory_map)
        costs.append(np.mean([bitstring_cut_weight(list(b), graph) for b in bitstrings]))

    return np.array(costs).reshape(width, width), program, native_program

## Choose a device

We choose the test device, initially as a QVM which can be run at any time but will not show the effects of diabling fencing on performance. The QVM supports the Quil-T interface to enable functional testing with the QVM prior to committing to a QPU reservation.

To run on a real QPU, obtain a reservation and update the name of the quantum computer to "Aspen-8".

In [ ]:
qc_name = "Aspen-8-qvm"

## Run landscape without global fencing disabled

First, we run and display the landscape produced without global fencing disabled. This is the default behavior of a Quil program.

In [ ]:
landscape_with_global_fencing, _, _ = run_maxcut_qaoa_landscape(
    qc_name=qc_name, disable_global_fencing=False
)
plot_landscape(landscape_with_global_fencing)

## Run landscape **with** global fencing disabled

Second, we run and display the landscape produced with global fencing disabled. Results on QPU show visually the improvement in fidelity that is gained by this parallel execution of 2Q gates.

In [ ]:
landscape_without_global_fencing, _, native_program = run_maxcut_qaoa_landscape(
    qc_name=qc_name, disable_global_fencing=True
)
plot_landscape(landscape_without_global_fencing)

To see what the native program looks like when fencing is disabled, you can print it out. When this is run on QPU, you will see "DEFCAL" instructions that include "FENCE" directives that only isolate the qubit pair being operated upon, whereas the default calibration contains "FENCE_ALL" directives.

In [ ]:
print(native_program)